# Asset Selection

In [1]:
import pandas as pd
import numpy as np

import scipy.stats as stats
from sklearn import covariance

import datetime as dt
import yfinance as yf

import matplotlib.pyplot as plt
import seaborn as sns

import pickle as pk
import requests

import random

jtplot.style(figsize=(15, 9))

In [3]:
# import bs4 as bs

# response = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
# soup = bs.BeautifulSoup(response.text)
# table = soup.find('table', {'class': 'wikitable sortable'})

# tickers = []
# for row in table.findAll('tr')[1:]:
#     ticker = row.findAll('td')[0].text.replace('\n', '')
#     if '.' in ticker:
#         ticker = ticker.replace('.', '-')
#     tickers.append(ticker)

# with open('../data/sp500tickers.pickle', 'wb') as f:
#     pk.dump(tickers, f)

# print('---------------')
# print(f'Ticker Amount: {len(tickers)}')
# print('---------------')

---------------
Ticker Amount: 503
---------------


In [2]:
with open('../data/sp500tickers.pickle', 'rb') as f:
    sp500 = pk.load(f)

with open('../../../alphavantage_token.TXT', 'rb') as f:
    TOKEN = f.read().decode('utf-8')

In [22]:
sp500[:10]

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']

In [3]:
d = {'Symbol': 'MMM', 'AssetType': 'Common Stock', 'Name': '3M Company',
     'Description': 'ood, a suburb of Saint Paul, Minnesota.',
     'CIK': '66740', 'Exchange': 'NYSE', 'Currency': 'USD',
     'Country': 'USA', 'Sector': 'LIFE SCIENCES',
     'Industry': 'SURGICAL & MEDICAL INSTRUMENTS & APPARATUS',
     'Address': '3M CENTER, BLDG. 220-13E-26A, ST PAUL, MN, US',
     'FiscalYearEnd': 'December', 'LatestQuarter': '2024-03-31',
     'MarketCapitalization': '57903698000', 'EBITDA': '8146000000',
     'PERatio': 'None', 'PEGRatio': '1.905', 'BookValue': '8.79',
     'DividendPerShare': '6.01', 'DividendYield': '0.0268',
     'EPS': '-12.73', 'RevenuePerShareTTM': '58.89',
     'ProfitMargin': '-0.216', 'OperatingMarginTTM': '0.217',
     'ReturnOnAssetsTTM': '0.0758', 'ReturnOnEquityTTM': '-0.693',
     'RevenueTTM': '32653001000', 'GrossProfitTTM': '15000000000',
     'DilutedEPSTTM': '-12.73', 'QuarterlyEarningsGrowthYOY': '-0.051',
     'QuarterlyRevenueGrowthYOY': '-0.003', 'AnalystTargetPrice': '110',
     'AnalystRatingStrongBuy': '0', 'AnalystRatingBuy': '5',
     'AnalystRatingHold': '11', 'AnalystRatingSell': '2',
     'AnalystRatingStrongSell': '0', 'TrailingPE': '-',
     'ForwardPE': '10.6', 'PriceToSalesRatioTTM': '1.546',
     'PriceToBookRatio': '10.51', 'EVToRevenue': '1.879',
     'EVToEBITDA': '-9.08', 'Beta': '1.001', '52WeekHigh': '105.31',
     '52WeekLow': '68.63', '50DayMovingAverage': '101.16',
     '200DayMovingAverage': '97.91', 'SharesOutstanding': '553361000',
     'DividendDate': '2024-06-12', 'ExDividendDate': '2024-05-23'}
d.keys()

dict_keys(['Symbol', 'AssetType', 'Name', 'Description', 'CIK', 'Exchange', 'Currency', 'Country', 'Sector', 'Industry', 'Address', 'FiscalYearEnd', 'LatestQuarter', 'MarketCapitalization', 'EBITDA', 'PERatio', 'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS', 'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM', 'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM', 'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'AnalystRatingStrongBuy', 'AnalystRatingBuy', 'AnalystRatingHold', 'AnalystRatingSell', 'AnalystRatingStrongSell', 'TrailingPE', 'ForwardPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue', 'EVToEBITDA', 'Beta', '52WeekHigh', '52WeekLow', '50DayMovingAverage', '200DayMovingAverage', 'SharesOutstanding', 'DividendDate', 'ExDividendDate'])

In [4]:
data = pd.read_csv('../data/sp500_Company_Overview.csv', index_col=0)
data.head()

,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
MMM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
for ticker in sp500[0:25]:
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={TOKEN}'
    r = requests.get(url)
    data.loc[ticker] = r.json()

data.head()

,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
MMM,Common Stock,3M Company,The 3M Company is an American multinational co...,66740,NYSE,USD,USA,LIFE SCIENCES,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,"3M CENTER, BLDG. 220-13E-26A, ST PAUL, MN, US",...,1.879,-9.08,1.001,105.52,68.63,101.5,98.13,553361000,2024-06-12,2024-05-23
AOS,Common Stock,Smith AO Corporation,A. O. Smith Corporation is an American manufac...,91142,NYSE,USD,USA,MANUFACTURING,HOUSEHOLD APPLIANCES,"11270 WEST PARK PLACE, MILWAUKEE, WI, US",...,3.248,15.19,1.141,92.44,63.34,83.92,81.37,120784000,2024-08-15,2024-07-31
ABT,Common Stock,Abbott Laboratories,Abbott Laboratories is an American multination...,1800,NYSE,USD,USA,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,"100 ABBOTT PARK ROAD, ABBOTT PARK, IL, US",...,4.928,18.75,0.718,120.4,88.33,103.84,106.6,1739630000,2024-08-15,2024-07-15
ABBV,Common Stock,AbbVie Inc,AbbVie is an American publicly traded biopharm...,1551152,NYSE,USD,USA,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,"1 NORTH WAUKEGAN ROAD, NORTH CHICAGO, IL, US",...,6.15,19.45,0.623,179.55,132.11,166.45,162.53,1765870000,2024-08-15,2024-07-15
ACN,Common Stock,Accenture plc,Accenture plc is an Irish-domiciled multinatio...,1467373,NYSE,USD,USA,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC","1 GRAND CANAL SQUARE, GRAND CANAL HARBOUR, DUB...",...,3.041,18.21,1.227,384.33,277.47,301.44,329.36,626384000,2024-08-15,2024-07-11


In [20]:
data['Beta'] = data['Beta'].astype(float)


In [45]:
# data[data['Beta']>1].groupby('Sector').count()

In [7]:
data.to_csv('../data/sp500_Company_Overview.csv')